In [2]:
# -*- coding:utf-8 -*-
# Parsing dividends data from DART

import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
import getopt
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import parser
import pandas as pd
from collections import defaultdict
import numpy as np

In [3]:
# 제일 파이썬 답다!
def get_DATA2(k, sample_dict, col_nm): 
    """ 딕셔너리로 col name 찾기 """

    col_dict = {
        'AADJ2':['자산총계','총자산','銃자산'],
        'AADJ8':['유동자산'],
        'AADJ4':['자본총계','총자본','銃자본'],
        'AADJ6':['유동부채'],
        'AADJ7':['비유동부채', '고정부채'],
        'AADJ5':['매출액']
    }

    col_kor_nm = col_dict.get(col_nm)

    for kor in col_kor_nm:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor, ['','',''])  #defualt values = ['','','']
            break
        else: 
            out = None
        
    return out


In [5]:
def cleansing(text):
    pattern = r'quot'  # 일본어,한자제거  quot
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'([ぁ-ゟ゠-ヿ])'  # 일본어제거, 一-鿋 한자제거 
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'\xa0'  # 줄바꿈제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'([ㄱ-ㅎㅏ-ㅣ])'  # 한글자음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[^\w\s]'  # 특수기호 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[\Ⅰ\Ⅱ\ㆍ]'  # 로마자 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[0-9]'  # 아라비아숫자 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r' '  # 띄어쓰기 제거
    text = re.sub(pattern=pattern, repl='', string=text)

    return text

In [ ]:
# url 읽기 dart2, dart3

if dart3:
    link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=2&offset=4916&length=3668&dtd=dart3.xsd"
    #link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=dart3.xsd"
    
elif (title == "분기ㆍ중간배당결정"):
    link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=dart2.dtd"
    
elif dart2:
    if year < 2007:
        link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=dart2.dtd"
    else:
        link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=87&offset=7601&length=4738&dtd=dart2.dtd"
        
else:
    link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=HTML"  


In [ ]:
# Don't compare boolean values to True or False using ==.

if not target['기재정정']:  #if target['기재정정'] == 0
    eleId = '8'  # 재무제표 8
else:
    eleId = '9'  # 재무제표 8

    

In [569]:
rpt_nm = {
    '합병등종료보고서(자산양수도)' : ['합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(자산양수도)'],  # 8,9
    '합병등종료보고서(영업양수도)': ['합병등종료보고서(영업양수도)', '[기재정정]합병등종료보고서(영업양수도)'],  # 8,9
    '자산양수ㆍ도종료보고서' : ['자산양수ㆍ도종료보고서', '[기재정정]자산양수ㆍ도종료보고서'],  # 3,4
    '영업양수ㆍ도종료보고서' : ['영업양수ㆍ도종료보고서', '[기재정정]영업양수ㆍ도종료보고서']  # 4,5
}

df_ASSET_TRANSFER = pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')

# 다른 문서 함께 처리할 시 씹혀버리는 문제 발생 

def report_type_for_eleId(rpt_nm):

    if rpt_nm in ['합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(자산양수도)',
                  '합병등종료보고서(영업양수도)', '[기재정정]합병등종료보고서(영업양수도)'] : 
        eleId = 8
    elif rpt_nm in ['자산양수ㆍ도종료보고서', '[기재정정]자산양수ㆍ도종료보고서'] : 
        eleId = 3
    elif rpt_nm in ['영업양수ㆍ도종료보고서', '[기재정정]영업양수ㆍ도종료보고서'] :
        eleId = 4
    else: 
        eleId = None
    
#     if rpt_nm in ['[기재정정]자산양수ㆍ도종료보고서', '[기재정정]합병등종료보고서(자산양수도)', 
#                   '[기재정정]합병등종료보고서(영업양수도)', '[기재정정]영업양수ㆍ도종료보고서']: 
#         eleId = eleId + 1
    
    return str(eleId)


df_ASSET_TRANSFER.rpt_nm.apply(lambda x: report_type_for_eleId(x)).value_counts()



3    981
8    961
4     71
Name: rpt_nm, dtype: int64

In [76]:
20070928000333


<iframe id="ifrm" frameborder="0" scrolling="auto" style="width: 100%; margin: 0px; padding: 0px; height: 676px;" title="tailFrame" 
src="/report/viewer.do?rcpNo=20070928000333&amp;dcmNo=1723800&amp;eleId=103&amp;offset=6710&amp;length=2310&amp;dtd=dart2.dtd">
</iframe>

link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=87&offset=7601&length=4738&dtd=dart2.dtd"
link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=dart2.dtd"

//*[@id="ext-gen10"]/div/li[4]/div/a/span


20070611000276

<iframe id="ifrm" frameborder="0" scrolling="auto" style="width: 100%; margin: 0px; padding: 0px; height: 676px;" title="tailFrame" 
src="/report/viewer.do?rcpNo=20070611000276&amp;dcmNo=1659323&amp;eleId=101&amp;offset=7263&amp;length=1794&amp;dtd=dart2.dtd">
</iframe>

//*[@id="ext-gen10"]/div/li[4]/div/a/span


NameError: name 'link2' is not defined

In [196]:
# 출처: https://rednooby.tistory.com/104 [개발자의 취미생활]


# import unicodedata
# import json
# type(test[-2])
# type(test[-2].string)
# # # 1. 자산양수ㆍ도 일정
# # str(test[-2]).split('// ')[5]
# # # 2. 자산양수ㆍ도승인주주총회 결과요약
# # str(test[-2]).split('// ')[6]
# # "3. 자산양수ㆍ도 전ㆍ후의 자산 증가(감소)내역"
# str(test[-2]).split('// ')[7]
# # # 4. 기타사항
# # str(test[-2]).split('// ')[8]


# for string in test[-2].stripped_strings:
#     textValue = string.split('// ')[7].encode('utf-8')
#     print(type(textValue))
#     print(textValue)




# # 출처: https://pyther.tistory.com/11 [pyther]

# type(test[-2].string)
# test[-2].string
# test[-2].stripped_strings

# for string in test[-2].stripped_strings:
#     print(type(string))
# #     print(string)
#     print(string.split('// 4')[1])
# # string


# treeNode1 = u'''
#             new Tree.TreeNode({
#                 text: "111",
#                 id: "101",
#                 cls: "text",
#                 listeners: {
#                     click: function() {viewDoc('20070611000276', '1659323', '101', '7263', '1794', 'dart2.dtd');}
#                 }
#             });
#             '''

# textValue = treeNode1
# jsonValue = '{%s}' % (textValue.split('{', 1)[1].rsplit('}', 1)[0],)
# value = json.loads(jsonValue)

<class 'bs4.element.ResultSet'>
10
<script type="text/javascript">

var winTestNotice = null;	// xwindow object
var divTestNotice = "divTestNoticeWin";	// ajax 수신 div
$j(document).ready(function() {
	initPage();

	initScroll();
	$j(window).resize(function(){
		$j("#center-panel > div > div").resize(resizeFrame);
		resizeFrame();
	}).resize();
});


var resizeFrame = function() {
	w	= parseInt($j("#center-panel > div > div").css("width"));
	h	= parseInt($j("#center-panel > div > div").css("height"));
	//$j("#ifrm").css("width", (w - 12) + "px");
	
		
	
	$j("#ifrm").css("height", h + "px");
	$j("#mobileScroll").css("height", h + "px");
};

var west;

//모바일(ios)에서 본문내용 스크롤 추가
function initScroll() {
	var iosKeywords = new Array("iPhone","iPod","iPad");
	for(var i=0; i<iosKeywords.length; i++){
		if(navigator.userAgent.indexOf(iosKeywords[i]) > -1) {
			$j("#mobileScroll").css("overflow","auto");
			$j("#mobileScroll").css("-webkit-overflow-scrolling","touch");
			$j("#mobileScroll").css("

In [574]:
df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER.eleId=='4'].rcp_no.apply(lambda x:str(x)[0:4]).value_counts()

2005    20
2008    17
2007    15
2006    15
2009     4
Name: rcp_no, dtype: int64

In [573]:
df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER.eleId=='3'].rcp_no.apply(lambda x:str(x)[0:4]).value_counts()

2007    266
2008    265
2006    261
2005    154
2009     34
2010      1
Name: rcp_no, dtype: int64

In [571]:
df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER.eleId=='8'].rcp_no.apply(lambda x:str(x)[0:4]).value_counts()

2009    124
2015    116
2018    110
2017    103
2016    100
2010     95
2014     77
2012     72
2011     63
2013     58
2019     43
Name: rcp_no, dtype: int64

In [588]:
df_ASSET_TRANSFER = pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')

df_ASSET_TRANSFER['url'] = df_ASSET_TRANSFER.rcp_no.apply(lambda x: "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(x))
df_ASSET_TRANSFER['eleId'] = df_ASSET_TRANSFER.rpt_nm.apply(lambda x: report_type_for_eleId(x))
df_sample = df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER['삭제'].isnull()].loc[df_ASSET_TRANSFER.eleId.isin(['3'])]
# df_sample= df_sample.iloc[:50,:]

display(df_sample.head(200))


,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,eleId
1,NaN,E003,Y,KR모터스,A000040,자산양수ㆍ도종료보고서,20070928000333,KR모터스,20070928,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
10,NaN,E003,Y,유유제약,A000220,자산양수ㆍ도종료보고서,20070611000276,유유제약,20070611,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
11,NaN,E003,K,삼천당제약,A000250,자산양수ㆍ도종료보고서,20060807000100,삼천당제약,20060807,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
12,NaN,E003,Y,대유플러스,A000300,자산양수ㆍ도종료보고서,20060831000095,대유플러스,20060831,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
13,NaN,E003,Y,노루홀딩스,A000320,자산양수ㆍ도종료보고서,20050718000075,노루홀딩스,20050718,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
14,NaN,E003,Y,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080526000241,한화손해보험,20080526,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
15,NaN,E003,Y,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080617000073,한화손해보험,20080617,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
19,NaN,E003,Y,한국주철관공업,A000970,자산양수ㆍ도종료보고서,20070817000105,한국주철관공업,20070817,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
34,NaN,E003,Y,CJ,A001040,자산양수ㆍ도종료보고서,20080918000115,CJ,20080918,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
43,NaN,E003,Y,대한방직,A001070,자산양수ㆍ도종료보고서,20051107000156,대한방직,20051107,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3


In [587]:
len(df_sample)

450

In [585]:
# 영업양수도 "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20070611000276"
# 20060807000100

def yo(rcpNo):
    handle = urllib.request.urlopen(f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}")
    data = handle.read()
    soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
    test = soup2.findAll('script', {'type': 'text/javascript'})
    matched = re.search('viewDoc(.*)', test[-2].string.split('// ')[7].replace("\n","").replace("\t",""),re.S) # 기재정정은 8 
    print(matched)
    words = matched.group(1).replace(');}}});cnt++;treeRoot.appendChild(treeNode1);', '').replace('(',"").replace("'",'').split(',')
    return words

df_sample['dcm_no']= df_sample.rcp_no.apply(lambda x: yo(x)[1])
df_sample['eleId1']= df_sample.rcp_no.apply(lambda x: yo(x)[2])
df_sample['end']= df_sample.rcp_no.apply(lambda x: yo(x)[-1])

# 영업양수도
# ['20070928000333', ' 1723800', ' 103', ' 6710', ' 2310', ' dart2.dtd']
# ['20070611000276', ' 1659323', ' 101', ' 7263', ' 1794', ' dart2.dtd']
# ['20060807000100', ' 1422482', ' 79', ' 4852', ' 2344', ' dart2.dtd']
# ['20060831000095', ' 1436689', ' 79', ' 4761', ' 2835', ' dart2.dtd']
# ['20050718000075', ' 1139516', ' 80', ' 5298', ' 2543', ' dart2.dtd']


# src="/report/viewer.do?rcpNo=20070928000333&amp;dcmNo=1723800&amp;eleId=103&amp;offset=6710&amp;length=2310&amp;dtd=dart2.dtd">
# src="/report/viewer.do?rcpNo=20070611000276&amp;dcmNo=1659323&amp;eleId=101&amp;offset=7263&amp;length=1794&amp;dtd=dart2.dtd">
# http://dart.fss.or.kr/report/viewer.do?rcpNo=20050119000173&dcmNo=976094&eleId=163&offset=7601&length=4738&dtd=dart2.dtd

<_sre.SRE_Match object; span=(120, 236), match="viewDoc('20070928000333', '1723800', '103', '6710>
<_sre.SRE_Match object; span=(120, 236), match="viewDoc('20070611000276', '1659323', '101', '7263>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060807000100', '1422482', '79', '4852'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060831000095', '1436689', '79', '4761'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20050718000075', '1139516', '80', '5298'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080526000241', '1951553', '91', '5880'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080617000073', '1964064', '92', '5882'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20070817000105', '1702411', '79', '4841'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080918000115', '2022887', '79', '4639'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20051107000156', '1196420', '86', '4814'>
<_sre.SRE_

<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20061013000064', '1454755', '79', '4642'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20071221000101', '1774426', '79', '4649'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20090106000190', '2083826', '84', '4681'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20061018000063', '1456644', '87', '4652'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20070215000065', '1517374', '86', '4647'>
<_sre.SRE_Match object; span=(120, 236), match="viewDoc('20070521000130', '1647969', '107', '8143>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20070423000304', '1612597', '79', '4865'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080516000260', '1946401', '79', '4630'>
<_sre.SRE_Match object; span=(119, 233), match="viewDoc('20050217000258', '989042', '90', '5174',>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20081001000280', '2029677', '84', '4898'>
<_sre.SRE_

<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060112000021', '1239233', '99', '6677'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060831000082', '1436660', '79', '4686'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060517000069', '1385601', '79', '4782'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20081114002024', '2057097', '80', '4585'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080222000374', '1808903', '79', '4515'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20050607000181', '1117870', '86', '4765'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080813000283', '1998946', '80', '4613'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20050520000016', '1110140', '94', '5863'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20081024000144', '2041458', '85', '4662'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20060323000030', '1294495', '86', '4698'>
<_sre.SRE_

<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20070912000260', '1716571', '86', '4746'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20061207000080', '1481723', '79', '4634'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20071218000250', '1773104', '80', '4706'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080724000200', '1987066', '80', '4887'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080911000154', '2020227', '90', '4783'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20061020000083', '1457908', '80', '4659'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080611000305', '1961283', '80', '4614'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20050621000140', '1125132', '96', '5632'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20061226000185', '1491568', '84', '5202'>
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20080625000260', '1969117', '81', '4839'>
<_sre.SRE_

KeyboardInterrupt: 

In [586]:
df_sample

,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,eleId
1,NaN,E003,Y,KR모터스,A000040,자산양수ㆍ도종료보고서,20070928000333,KR모터스,20070928,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
10,NaN,E003,Y,유유제약,A000220,자산양수ㆍ도종료보고서,20070611000276,유유제약,20070611,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
11,NaN,E003,K,삼천당제약,A000250,자산양수ㆍ도종료보고서,20060807000100,삼천당제약,20060807,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
12,NaN,E003,Y,대유플러스,A000300,자산양수ㆍ도종료보고서,20060831000095,대유플러스,20060831,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
13,NaN,E003,Y,노루홀딩스,A000320,자산양수ㆍ도종료보고서,20050718000075,노루홀딩스,20050718,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
14,NaN,E003,Y,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080526000241,한화손해보험,20080526,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
15,NaN,E003,Y,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080617000073,한화손해보험,20080617,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
19,NaN,E003,Y,한국주철관공업,A000970,자산양수ㆍ도종료보고서,20070817000105,한국주철관공업,20070817,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
34,NaN,E003,Y,CJ,A001040,자산양수ㆍ도종료보고서,20080918000115,CJ,20080918,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3
43,NaN,E003,Y,대한방직,A001070,자산양수ㆍ도종료보고서,20051107000156,대한방직,20051107,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,3


In [484]:
# 영업양수도 "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20070611000276"
# 20060807000100


handle = urllib.request.urlopen("http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20050718000075")
# print(f'여기다!!!{i}', handle)
data = handle.read()
soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
# print(soup2)

test = soup2.find('a', {'href': '#download'})['onclick']
# print(test)

test = soup2.findAll('script', {'type': 'text/javascript'})

# 출처: https://rednooby.tistory.com/104 [개발자의 취미생활]



print(test[-2].string.split('// ')[7].replace("\n","").replace("\t",""))

matched = re.search('viewDoc(.*)', test[-2].string.split('// ')[7].replace("\n","").replace("\t",""),re.S)
print(matched)

words = matched.group(1).replace(');}}});cnt++;treeRoot.appendChild(treeNode1);', '').replace('(',"").replace("'",'').split(',')
print(words)

words[0]

# 영업양수도
# ['20070928000333', ' 1723800', ' 103', ' 6710', ' 2310', ' dart2.dtd']
# ['20070611000276', ' 1659323', ' 101', ' 7263', ' 1794', ' dart2.dtd']
# ['20060807000100', ' 1422482', ' 79', ' 4852', ' 2344', ' dart2.dtd']
# ['20060831000095', ' 1436689', ' 79', ' 4761', ' 2835', ' dart2.dtd']
# ['20050718000075', ' 1139516', ' 80', ' 5298', ' 2543', ' dart2.dtd']


4treeNode1 = new Tree.TreeNode({text: "3. 자산양수ㆍ도 전ㆍ후의 자산 증가(감소)내역",id: "80",cls: "text",listeners: {click: function() {viewDoc('20050718000075', '1139516', '80', '5298', '2543', 'dart2.dtd');}}});cnt++;treeRoot.appendChild(treeNode1);
<_sre.SRE_Match object; span=(119, 234), match="viewDoc('20050718000075', '1139516', '80', '5298'>
['20050718000075', ' 1139516', ' 80', ' 5298', ' 2543', ' dart2.dtd']


'20050718000075'

In [581]:
df_sample[['crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'dcm_no','eleId1','eleId','end']]

,crp_nm,crp_cd,rpt_nm,rcp_no,dcm_no,eleId1,eleId,end
1,KR모터스,A000040,자산양수ㆍ도종료보고서,20070928000333,1723800,103,3,dart2.dtd
10,유유제약,A000220,자산양수ㆍ도종료보고서,20070611000276,1659323,101,3,dart2.dtd
11,삼천당제약,A000250,자산양수ㆍ도종료보고서,20060807000100,1422482,79,3,dart2.dtd
12,대유플러스,A000300,자산양수ㆍ도종료보고서,20060831000095,1436689,79,3,dart2.dtd
13,노루홀딩스,A000320,자산양수ㆍ도종료보고서,20050718000075,1139516,80,3,dart2.dtd
14,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080526000241,1951553,91,3,dart2.dtd
15,한화손해보험,A000370,자산양수ㆍ도종료보고서,20080617000073,1964064,92,3,dart2.dtd
19,한국주철관공업,A000970,자산양수ㆍ도종료보고서,20070817000105,1702411,79,3,dart2.dtd
34,CJ,A001040,자산양수ㆍ도종료보고서,20080918000115,2022887,79,3,dart2.dtd
43,대한방직,A001070,자산양수ㆍ도종료보고서,20051107000156,1196420,86,3,dart2.dtd


In [512]:
# 합병등종료보고서


handle = urllib.request.urlopen("http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171113000213")
# print(f'여기다!!!{i}', handle)
data = handle.read()
soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
# print(soup2)

test = soup2.find('a', {'href': '#download'})['onclick']
# print(test)

test = soup2.findAll('script', {'type': 'text/javascript'})
print(test[-2])
# 출처: https://rednooby.tistory.com/104 [개발자의 취미생활]



print(test[-2].string.split('//'))

# matched = re.search('viewDoc(.*)', test[-2].string.split('// ')[11].replace("\n","").replace("\t",""),re.S)
# print(matched)

# words = matched.group(1).replace("west.setRootNodetreeRoot);var viewport = new Ext.Viewport{     layout:border', '     items:[     north', '     west', '     new Ext.Panel{             region:center', '             id:center-panel', '             margins:5 5 5 5', '             autoScroll:false', '             items: [{                 contentEl: center', '                 border:false', '                 autoScroll:false             }]         })      ] });if cnt > 200) {treeRoot.expand);} else {west.expandAll);} $j","")
# print(words)

# 20171113000213', '5840458', '8', '7805', '4194', 'dart3.xsd')
 "&offset=4916&length=3668&dtd=dart3.xsd"

<script type="text/javascript">

var winTestNotice = null;	// xwindow object
var divTestNotice = "divTestNoticeWin";	// ajax 수신 div
$j(document).ready(function() {
	initPage();

	initScroll();
	$j(window).resize(function(){
		$j("#center-panel > div > div").resize(resizeFrame);
		resizeFrame();
	}).resize();
});


var resizeFrame = function() {
	w	= parseInt($j("#center-panel > div > div").css("width"));
	h	= parseInt($j("#center-panel > div > div").css("height"));
	//$j("#ifrm").css("width", (w - 12) + "px");
	
		
	
	$j("#ifrm").css("height", h + "px");
	$j("#mobileScroll").css("height", h + "px");
};

var west;

//모바일(ios)에서 본문내용 스크롤 추가
function initScroll() {
	var iosKeywords = new Array("iPhone","iPod","iPad");
	for(var i=0; i<iosKeywords.length; i++){
		if(navigator.userAgent.indexOf(iosKeywords[i]) > -1) {
			$j("#mobileScroll").css("overflow","auto");
			$j("#mobileScroll").css("-webkit-overflow-scrolling","touch");
			$j("#mobileScroll").css("width","100%");
		}
	}
}

function 

In [485]:


def main():
    
    # 어쩌고 데이터프레임 
    
    df_ASSET_TRANSFER = pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')
    
    df_ASSET_TRANSFER['url'] = df_ASSET_TRANSFER.rcp_no.apply(lambda x: "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(x))
    df_ASSET_TRANSFER['eleId'] = df_ASSET_TRANSFER.rpt_nm.apply(lambda x: report_type_for_eleId(x))
    
#     df_ASSET_TRANSFER['기재정정'] = df_ASSET_TRANSFER['rpt_nm'].isin(
#         ['[기재정정]자산양수ㆍ도종료보고서', '[기재정정]합병등종료보고서(자산양수도)', 
#          '[기재정정]합병등종료보고서(영업양수도)', '[기재정정]영업양수ㆍ도종료보고서'] 
#     ).apply(int)
    
#     rpt_nm_list = [
#         ['합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(자산양수도)'],  # 8,9
#         ['합병등종료보고서(영업양수도)', '[기재정정]합병등종료보고서(영업양수도)'],  # 8,9
#         ['영업양수ㆍ도종료보고서', '[기재정정]영업양수ㆍ도종료보고서'],  # 4,5
#         ['자산양수ㆍ도종료보고서', '[기재정정]자산양수ㆍ도종료보고서']  # 3,4
#     ]

    df_sample = df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER['삭제'].isnull()].loc[df_ASSET_TRANSFER.eleId=='8']
#     .loc[
#         df_ASSET_TRANSFER.rpt_nm.isin(rpt_nm_list[0])]
    #df_sample = df_sample.loc[df_sample.index.isin([0,3,16,17,18])]
    df_sample= df_sample.iloc[:50,:]
    
    display(df_sample.head(20))
    
    
    ###### 돌리기~~~~~
    dart_div_list = list()

    for i in range(0, 20):
        target = df_sample.iloc[i]
        # print(target)
        crpCd = target.crp_cd
        crpNm = target.crp_nm
        rcpDt = target.rcp_dt
        link = target.url
        eleId = target.eleId
    
        handle = urllib.request.urlopen(link)
        print(f'여기다!!!{i}', handle)
        data = handle.read()
        soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
        # print(soup2)

        test = soup2.find('a', {'href': '#download'})['onclick']
        # print(test)

        words = test.split("'")
        # print(words)

        rcpNo = words[1]
        dcmNo = words[3]
        # print(rcpNo)
        # print(dcmNo)

        dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
        # print(dart2)
        dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
        # print(dart3)


            
        ####### 뷰어 링크
        if dart3:
            link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=" + eleId + "&offset=4916&length=3668&dtd=dart3.xsd"
        print(link2)
        

        
        ####### 테이블 찾기
        handle = urllib.request.urlopen(link2)
        # print(handle)
        data = handle.read()
        soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
        # print(soup3)

        tables = soup3.findAll("table")  # list of the all tables in the link2 which means number of tables
        print('len(tables):', len(tables))  # length of the list

        
        if not tables:
            print('해당사항없음, 판단 -> n_tables 컬럼 ')
            # continue 하게 되면. 누락문제발생
            continue
        elif len(tables) == 1:  # 재무제표만
            div_table = soup3.find("table")
        elif 2 <= len(tables) <= 6:  # 단위, 재무제표
            div_table = soup3.findAll("table")[1]
        else: 
            print('일단 한번 보자, 이후에 이유쓰는 란 추가 ')
            break
            #continue ??? 아닌가 ? 밀려있는지 확인필요함

        # print(div_table)  # 예외처리 후 한가지의 table 파싱

        
        # rows
        div_trs = div_table.findAll('tr')  # list of the all tr in that tables 
        print('len(div_trs)',len(div_trs))  # the length of the rows
        #print(div_trs)
        
        
        # columns -> 4개 ok, 2,5개 예외처리 해야함
        div_tds = div_trs[1]  # list of the all td in that tables
        print('len(div_tds)',len(div_tds))  # the number of columns 
        #print(div_tds)
        
        
        
        
        
        ####### 딕셔너리에 넣기
        dict1 = defaultdict(list)

        for i in range(len(div_trs)):
            for j in range(1, 4):
                try:
                    key = cleansing(div_trs[i].findAll('td')[0].text.strip())
                    val = div_trs[i].findAll('td')[j].text.strip()
                    dict1[key].append(val)
                except IndexError:
                    pass
                
        #print(dict(dict1))
        dict1['STK_CD'].append(crpCd)
        dict1['STK_NM_KOR'].append(crpNm)
        dict1['rcp_NO'].append(rcpNo)
        dict1['rcp_DT'].append(rcpDt)
        dict1['url'].append(link)

        dart_div_list.append(dict(dict1))

    print("")
    print("")
    print('len(dart_div_list)', len(dart_div_list))
    #print(dart_div_list)

    
    ##### 엑셀에 저장하기 
    cur_dir = os.getcwd()

    workbook_name = "DART_dividends_baseline_1.xlsx"

    workbook = xlsxwriter.Workbook(workbook_name)

    #포멧지정
    worksheet_result = workbook.add_worksheet('result')
    filter_format = workbook.add_format({'bold': True,
                                         'fg_color': '#D7E4BC'
                                         })

    percent_format = workbook.add_format({'num_format': '0.00%'})

    roe_format = workbook.add_format({'bold': True,
                                      'underline': True,
                                      'num_format': '0.00%'})

    num_format = workbook.add_format({'num_format': '0.00'})
    num2_format = workbook.add_format({'num_format': '#,##0'})
    num3_format = workbook.add_format({'num_format': '#,##0.00',
                                       'fg_color': '#FCE4D6'})

    worksheet_result.set_column('A:A', 10)
    worksheet_result.set_column('B:B', 10)
    worksheet_result.set_column('C:C', 10)
    worksheet_result.set_column('D:D', 10)
    worksheet_result.set_column('H:H', 10)
    worksheet_result.set_column('I:I', 10)
    worksheet_result.set_column('J:J', 10)
    worksheet_result.set_column('K:K', 10)
    worksheet_result.set_column('L:L', 10)
    worksheet_result.set_column('M:M', 10)
    worksheet_result.set_column('N:N', 10)
    worksheet_result.set_column('O:O', 10)
    worksheet_result.set_column('P:P', 10)

    worksheet_result.write(0, 0, "STK_CD", filter_format)
    worksheet_result.write(0, 1, "STK_NM_KOR", filter_format)
    worksheet_result.write(0, 2, "rcp_NO", filter_format)
    worksheet_result.write(0, 3, "rcp_DT", filter_format)

    worksheet_result.write(0, 4, "자산_양수도전_자산총계", filter_format)
    worksheet_result.write(0, 5, "자산_증가감소_자산총계", filter_format)
    worksheet_result.write(0, 6, "자산_양수도후_자산총계", filter_format)

    worksheet_result.write(0, 7, "자산_양수도전_유동자산", filter_format)
    worksheet_result.write(0, 8, "자산_증가감소_유동자산", filter_format)
    worksheet_result.write(0, 9, "자산_양수도후_유동자산", filter_format)

    worksheet_result.write(0, 10, "자산_양수도전_자본총계", filter_format)
    worksheet_result.write(0, 11, "자산_증가감소_자본총계", filter_format)
    worksheet_result.write(0, 12, "자산_양수도후_자본총계", filter_format)

    worksheet_result.write(0, 13, "자산_양수도전_유동부채", filter_format)
    worksheet_result.write(0, 14, "자산_증가감소_유동부채", filter_format)
    worksheet_result.write(0, 15, "자산_양수도후_유동부채", filter_format)

    worksheet_result.write(0, 16, "자산_양수도전_비유동부채", filter_format)
    worksheet_result.write(0, 17, "자산_증가감소_비유동부채", filter_format)
    worksheet_result.write(0, 18, "자산_양수도후_비유동부채", filter_format)

    worksheet_result.write(0, 19, "url", filter_format)
    worksheet_result.write(0, 20, "n_tables", filter_format)
    worksheet_result.write(0, 21, "n_trs", filter_format)    
    worksheet_result.write(0, 21, "n_tds", filter_format)    

    
    ##### 데이터 작성하기 
    for k in range(len(dart_div_list)):

        STK_CD = dart_div_list[k].get('STK_CD')[0]  # 기업코드
        STK_NM_KOR = dart_div_list[k].get('STK_NM_KOR')[0]  #기업명
        rcp_NO = dart_div_list[k].get('rcp_NO')[0]  # 보고서번호
        rcp_DT = dart_div_list[k].get('rcp_DT')[0]  # 보고기일
        url = dart_div_list[k].get('url')[0]  # 보고서url

        # 여기 수정 들어감
        AADJ2 = get_DATA(k, dart_div_list, 'AADJ2')
        AADJ8 = get_DATA(k, dart_div_list, 'AADJ8')
        AADJ4 = get_DATA(k, dart_div_list, 'AADJ4')
        AADJ6 = get_DATA(k, dart_div_list, 'AADJ6')
        AADJ7 = get_DATA(k, dart_div_list, 'AADJ7')


        asset_before_AADI2 = AADJ2[0]  # 자산_양수도전_자산총계
        asset_increase_AADI2 = AADJ2[1]  # 자산_증가감소_자산총계
        asset_after_AADJ2 = AADJ2[2]  # 자산_양수도후_자산총계

        asset_before_AADI8 = AADJ8[0]  # 자산_양수도전_유동자산
        asset_increase_AADI8 = AADJ8[1]  # 자산_증가감소_유동자산
        asset_after_AADJ8 = AADJ8[2]  # 자산_양수도후_유동자산

        asset_before_AADI4 = AADJ4[0]  # 자산_양수도전_자본총계
        asset_increase_AADI4 = AADJ4[1]  # 자산_증가감소_자본총계
        asset_after_AADJ4 = AADJ4[2]  # 자산_양수도후_자본총계

        asset_before_AADI6 = AADJ6[0]  # 자산_양수도전_유동부채
        asset_increase_AADI6 = AADJ6[1]  # 자산_증가감소_유동부채
        asset_after_AADJ6 = AADJ6[2]  # 자산_양수도후_유동부채

        asset_before_AADI7 = AADJ7[0]  # 자산_양수도전_비유동부채
        asset_increase_AADI7 = AADJ7[1]  # 자산_증가감소_비유동부채
        asset_after_AADJ7 = AADJ7[2]  # 자산_양수도후_비유동부채


        worksheet_result.write(k + 1, 0, STK_CD)
        worksheet_result.write(k + 1, 1, STK_NM_KOR)
        worksheet_result.write(k + 1, 2, rcp_DT)
        worksheet_result.write(k + 1, 3, rcp_NO)

        worksheet_result.write(k + 1, 4, asset_before_AADI2)
        worksheet_result.write(k + 1, 5, asset_increase_AADI2)
        worksheet_result.write(k + 1, 6, asset_after_AADJ2)

        worksheet_result.write(k + 1, 7, asset_before_AADI8)
        worksheet_result.write(k + 1, 8, asset_increase_AADI8)
        worksheet_result.write(k + 1, 9, asset_after_AADJ8)

        worksheet_result.write(k + 1, 10, asset_before_AADI4)
        worksheet_result.write(k + 1, 11, asset_increase_AADI4)
        worksheet_result.write(k + 1, 12, asset_after_AADJ4)

        worksheet_result.write(k + 1, 13, asset_before_AADI6)
        worksheet_result.write(k + 1, 14, asset_increase_AADI6)
        worksheet_result.write(k + 1, 15, asset_after_AADJ6)

        worksheet_result.write(k + 1, 16, asset_before_AADI7)
        worksheet_result.write(k + 1, 17, asset_increase_AADI7)
        worksheet_result.write(k + 1, 18, asset_after_AADJ7)

        worksheet_result.write(k + 1, 19, url)
        worksheet_result.write(k + 1, 20, len(tables))
        worksheet_result.write(k + 1, 21, len(div_trs))
        worksheet_result.write(k + 1, 22, len(div_tds))
        
    workbook.close()

    
# Main
if __name__ == "__main__":
    start= time.time()
    
    main()
    
    print(time.time()- start)

,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,eleId
0,NaN,E003,Y,동화약품,A000020,합병등종료보고서(자산양수도),20171113000213,동화약품,20171113,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
3,NaN,E003,Y,CJ대한통운,A000120,합병등종료보고서(자산양수도),20120102000266,CJ대한통운,20120102,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
4,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090410000957,하이트진로홀딩스,20090410,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
5,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090521000091,하이트진로홀딩스,20090521,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
6,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090624000313,하이트진로홀딩스,20090625,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
7,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090804000007,하이트진로홀딩스,20090804,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
8,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20120510001132,하이트진로홀딩스,20120510,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
16,NaN,E003,Y,천일고속,A000650,합병등종료보고서(자산양수도),20170920000395,천일고속,20170921,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
17,NaN,E003,Y,LS네트웍스,A000680,합병등종료보고서(자산양수도),20150723000300,LS네트웍스,20150723,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8
18,NaN,E003,Y,이화산업,A000760,합병등종료보고서(자산양수도),20140401001282,이화산업,20140401,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,8


여기다!!!0 <http.client.HTTPResponse object at 0x000001C020575C50>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20171113000213&dcmNo=5840458&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 14
len(div_tds) 9
여기다!!!1 <http.client.HTTPResponse object at 0x000001C01EEDF160>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20120102000266&dcmNo=3235174&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 2
len(div_trs) 27
len(div_tds) 9
여기다!!!2 <http.client.HTTPResponse object at 0x000001C0204CDDA0>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20090410000957&dcmNo=2225328&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 4
len(div_trs) 6
len(div_tds) 9
여기다!!!3 <http.client.HTTPResponse object at 0x000001C0204B5DD8>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20090521000091&dcmNo=2315033&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 4
len(div_trs) 4
len(div_tds) 9
여기다!!!4 <http.client.HTTPResponse object at 0x000001C0204E5AC8>
http://dart.fss.or